In [1]:
!pip install opendatasets --upgrade --quiet
import opendatasets as od
import pandas as pd
import datetime
import matplotlib.dates as mdates

dataset = 'https://www.kaggle.com/datasets/arunavakrchakraborty/covid19-twitter-dataset?resource=download&select=Covid-19+Twitter+Dataset+%28Aug-Sep+2020%29.csv'

od.download(dataset)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: tanpeiwen
Your Kaggle Key: ··········


100%|██████████| 58.0M/58.0M [00:00<00:00, 94.8MB/s]


In [2]:
df1 = pd.read_csv("/content/covid19-twitter-dataset/Covid-19 Twitter Dataset (Apr-Jun 2020).csv")
df2 = pd.read_csv("/content/covid19-twitter-dataset/Covid-19 Twitter Dataset (Aug-Sep 2020).csv")

df = pd.concat([df1, df2], ignore_index=True)
df = df.reset_index()
df.head()
df=df.iloc[:,[2, 6, 13, 7]]
# convert to ints

df["created_at"] = pd.to_datetime(df["created_at"])
df['created_at'] = df['created_at'].map(mdates.date2num)
df.dropna(inplace=True)

df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 555550 entries, 0 to 555555
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   created_at      555550 non-null  float64
 1   favorite_count  555550 non-null  float64
 2   compound        555550 non-null  float64
 3   retweet_count   555550 non-null  float64
dtypes: float64(4)
memory usage: 21.2 MB


In [8]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

x=df.iloc[:,: -1].sample(n=10000).values
y=df.iloc[:,-1].sample(n=10000).values
y=y.reshape(-1,1)

# Use MinMax scaling on X2 and X3 features
#scaler=MinMaxScaler()

#x_scale=scaler.fit_transform(x)
#y_scale=scaler.fit_transform(y)

#rbf = SVR(kernel='rbf', C=1, epsilon=0.01)
#rbf.fit(x_scale,y_scale)

grid = GridSearchCV(
estimator=SVR(kernel='rbf'),
param_grid={

'C': [1, 10, 100, 1000],

'epsilon': [0.0001, 0.001, 0.01, 0.1, 1, 10]
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)


grid.fit(x,y)


print("Best: %f using %s" % (grid.best_score_,grid.best_params_))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best: -211165135.588753 using {'C': 1000, 'epsilon': 10}


In [9]:
rbf = SVR(kernel='rbf', C=1, epsilon=0.01)
rbf.fit(x,y)
import pickle
pickle.dump(rbf, open("svr_model", 'wb'))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
# single prediction
print(rbf.predict(np.array([5556,7000,0.23]).reshape(1,-1)))

[27.50112178]
